In [23]:
import cv2
import os
import numpy as np # <-- Import numpy

# --- Configuration ---
DATASET_DIR = 'dataset'
IMG_SIZE = (100, 100) # Size for the model
# The poses we want to capture
POSES = ['Center', 'Looking_Left', 'Looking_Right', 'Looking_Up', 'Looking_Down', 'Smile']
# --- End Configuration ---

# --- NEW: Load *TWO* classifiers ---
try:
    frontal_face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    profile_face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_profileface.xml')
except Exception:
    print("Error loading Haar Cascade classifiers.")
    print("Please ensure OpenCV is installed correctly.")
    exit()

# Ask for the person's name
person_name = input("Enter the name of the person: ").strip()
if not person_name:
    print("Name cannot be empty. Exiting.")
    exit()

# Create the directory for this person's images
person_dir = os.path.join(DATASET_DIR, person_name)
os.makedirs(person_dir, exist_ok=True)
print(f"Directory created: {person_dir}")

# Start the webcam
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

print("\nStarting interactive data collection...")
print("A window will open. Follow the instructions on the screen.")

# Loop through each pose
pose_index = 0
while pose_index < len(POSES):
    pose_name = POSES[pose_index]
    
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to capture frame.")
        break
        
    # Make copies to draw on
    display_frame = frame.copy()
    
    # Convert to grayscale for detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # --- NEW: Detect both frontal and profile faces ---
    frontal_faces = frontal_face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5, minSize=(50, 50))
    profile_faces = profile_face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5, minSize=(50, 50))

    # Combine the results
    if len(frontal_faces) > 0 and len(profile_faces) > 0:
        faces = np.concatenate((frontal_faces, profile_faces))
    elif len(frontal_faces) > 0:
        faces = frontal_faces
    elif len(profile_faces) > 0:
        faces = profile_faces
    else:
        faces = []
    # --- End of NEW detection logic ---

    # --- Draw UI Text ---
    # Instructions
    text1 = f"Pose: {pose_name.replace('_', ' ')}"
    text2 = "Press 's' to save. Press 'q' to quit."
    cv2.putText(display_frame, text1, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
    cv2.putText(display_frame, text2, (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    
    # Progress
    progress_text = f"Image {pose_index + 1} of {len(POSES)}"
    cv2.putText(display_frame, progress_text, (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

    # Draw a green box around the *first* detected face
    if len(faces) > 0:
        (x, y, w, h) = faces[0]
        cv2.rectangle(display_frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
    
    # Show the frame
    cv2.imshow('Interactive Data Collector', display_frame)

    key = cv2.waitKey(1) & 0xFF

    # --- Handle Key Presses ---
    if key == ord('s'):
        # Check if a face was actually detected
        if len(faces) == 0:
            print("No face detected! Please try again.")
            continue
            
        # We only save the first detected face
        (x, y, w, h) = faces[0]
        
        # Crop the face from the *original color* frame
        face_crop_color = frame[y:y+h, x:x+w]
        
        # Resize to our standard size
        face_resized = cv2.resize(face_crop_color, IMG_SIZE)
        
        # Create the filename
        img_filename = f"{pose_index:02d}_{pose_name}.jpg" # e.g., "01_Looking_Left.jpg"
        img_path = os.path.join(person_dir, img_filename)
        
        # Save the image
        cv2.imwrite(img_path, face_resized)
        
        print(f"Saved: {img_path}")
        
        # Move to the next pose
        pose_index += 1
        
    elif key == ord('q'):
        print("Quitting early.")
        break

# --- Cleanup ---
cap.release()
cv2.destroyAllWindows()
if pose_index == len(POSES):
    print("Data collection complete for all poses!")
else:
    print("Data collection stopped early.")

Directory created: dataset\yogesh

Starting interactive data collection...
A window will open. Follow the instructions on the screen.
Saved: dataset\yogesh\00_Center.jpg
Saved: dataset\yogesh\01_Looking_Left.jpg
No face detected! Please try again.
Saved: dataset\yogesh\02_Looking_Right.jpg
Saved: dataset\yogesh\03_Looking_Up.jpg
Saved: dataset\yogesh\04_Looking_Down.jpg
Saved: dataset\yogesh\05_Smile.jpg
Data collection complete for all poses!
